# Number of vertical blocks chosen by the ILP for different objective functions w.r.t. alpha 

All vertical blocks with columns less than $\alpha$ are chosen as optimum. 
We want to see if vertical blocks are part of the optimal solution, in which case, fixing them
helps to reduce computational resources and to achieve the same solution (or close enough to the optimal value). 

We show this efect for all objective functions. 

1. count the number of vertical blocks in the optimal solution
2. count the number of vertical blocks in the MSA
3. compute the ratio (1)/(2)

Equivalently, we just need to 
1. count nodes used by all sequences in the final graph


In [1]:
import sys
sys.path.append("..")

import json
import seaborn as sns 
import pandas as pd

from pathlib import Path
from collections import defaultdict, namedtuple
# from src.graph.load_gfa import load_gfa
from src.graph.info_from_gfa import load_info_graph
from src.utils.params_from_path import params_from_path
from src.utils.info_from_exp import get_info_from_exp

from src.graph.load_gfa import load_gfa

import numpy as np
import matplotlib.pyplot as plt

In [2]:
ORDER_OBJ_FUNCTIONS = ["blocks", "weighted", "depth",]# "strings",]# "pen_strings"]

NSEQS = 50
NAME = f"{NSEQS}-sars-cov-2-ena"
PATH_SAVE = Path(f"../output-paper-nodes-used-by-all-sequences/sars-cov-2-{NSEQS}")
PATH_SAVE.mkdir(exist_ok=True, parents=True)

PATH_EXPERIMENTS=Path("/data/analysis-paper/paper-experiment-decomposition")

In [3]:
# OBJ_FUNCTIONS = ["blocks", "weighted", "depth", "strings","pen_strings"]
# OBJ_FUNCTIONS = ["nodes", "weighted", "depth", "strings","depth_and_len"]
MIN_LEN = [0, 20]
MIN_COVERAGE = [0.0, 0.3]
DECOMPOSITION = "complete"
# DECOMPOSITION = "row_maximal"

## Nodes used by all sequences

In [4]:
path_graph = Path(f"/data/analysis-paper/paper-experiment-decomposition/sars-cov-2-50-{DECOMPOSITION}/gfa-unchop/strings/penalization0-min_len0-min_coverage0-alpha61/50-sars-cov-2-ena.gfa")

In [5]:
load_info_graph(path_gfa=path_graph)

{'n_vertices': 976,
 'n_edges': 1408,
 'len_graph': 60765,
 'potential_seeds_len15': 53633,
 'potential_seeds_len20': 51425,
 'potential_seeds_len25': 49258,
 'nodes_depth_gt_011': 0.507172131147541,
 'nodes_depth_gt_015': 0.5040983606557377,
 'nodes_depth_gt_02': 0.4948770491803279,
 'nodes_depth_gt_025': 0.492827868852459,
 'nodes_depth_gt_03': 0.49077868852459017}

In [6]:
nodes, edges, paths, node_depth  = load_gfa(path_graph)


In [7]:
len(dict(filter(lambda x: x[1] == 1. , node_depth.items())))

229

In [8]:
list_path_gfa=list(Path(f"/data/analysis-paper/paper-experiment-decomposition/sars-cov-2-50-{DECOMPOSITION}/gfa-unchop/").rglob("*.gfa"))

In [9]:
info = []
for path_gfa in list_path_gfa:

    # print(path_gfa.stem)
    obj_function = path_gfa.parent.parent.stem

    penalization, min_len, min_coverage, alpha = str(path_gfa.parent).split("/")[-1].split("-")
    # path_gfa.glob("*json")


    penalization = int(penalization.replace("penalization",""))
    min_len = int(min_len.replace("min_len",""))
    min_coverage = min_coverage.replace("min_coverage","")
    alpha = int(alpha.replace("alpha",""))

    # print(penalization, min_len, min_coverage, alpha)

    # contar bloques verticales en cada solucion 
    # cargar cada json de los subMSA y sumar sus bloques verticales
    n_vertical_blocks = 0
    for submsa_solution in list(path_gfa.glob("*json")):
        
        with open(submsa_solution,"r") as fp:
            blocks_submsa = json.load(fp)
            vertical_blocks_submsa = [b for b in blocks_submsa if len(b[0]) == NSEQS]
            n_vertical_blocks += len(vertical_blocks_submsa)
    # print(obj_function, penalization, min_len, min_coverage, alpha, n_vertical_blocks)
    

    nodes, edges, paths, node_depth  = load_gfa(path_gfa)
    
    nodes_full_depth = len(dict(filter(lambda x: x[1] == 1. , node_depth.items())))


    info.append(
        dict(
            objective_function=obj_function,
            penalization=penalization,
            min_len=min_len,
            min_coverage=min_coverage,
            alpha=alpha,
            vertical_blocks_ilp=n_vertical_blocks,
            path=str(path_gfa),
            nodes_full_depth = nodes_full_depth,
        )
    )

In [10]:
data = pd.DataFrame(info)
data["objective_function"] = data["objective_function"].apply(lambda x: "blocks" if x=="nodes" else x)
data["objective_function"] = data["objective_function"].apply(lambda x: "pen_strings" if x=="depth_and_len" else x)

In [11]:
sns.set_theme(style="darkgrid", rc={"figure.figsize": (8,7)})
g_vertical_blocks = sns.lineplot(data=data.query(f"objective_function in {ORDER_OBJ_FUNCTIONS}"),
                x="alpha",
                y="nodes_full_depth",
                hue="objective_function",
                style="objective_function",
                markers=True,
                hue_order=ORDER_OBJ_FUNCTIONS,
                )

g_vertical_blocks.set_xlabel(f"{chr(945)}",fontsize=16)
g_vertical_blocks.set_ylabel("#Nodes used by all sequences",fontsize=16)

g_vertical_blocks.set_ylim([0,250])
# g_vertical_blocks.set_xlim([1,650])

decomposition = DECOMPOSITION.replace("_","-")
sns.move_legend(
    g_vertical_blocks, "lower center",
    bbox_to_anchor=(.5, 1), ncol=5,
    title=f"{decomposition} decomposition", 
    frameon=False,
)

plt.tight_layout()

plt.savefig(PATH_SAVE.joinpath(f"{NAME}-vertical-blocks-ilp-{DECOMPOSITION}.png"),dpi=300)

___